In [6]:
import pandas as pd
import json
from openai import OpenAI
import os
from tqdm.notebook import tqdm
from dotenv import load_dotenv


In [7]:
load_dotenv()

with open('govbr_scripts_chat_online.json') as f:
    data = json.load(f)

In [8]:
def generate_prompt(classe, resposta):
    prompt = f"""
Please generate a question that an user might be asking in order to receive the 'answer' or 'resposta' provided below.
The 'classe', or 'class' is also provided to help you out in figuring a proper question.

### Classe: {classe}

### Resposta: {resposta}

Answer directly with the question, don't say anything else.
    """
    return prompt

#os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()
model = "gpt-4-turbo"

In [22]:
dataset = pd.DataFrame(columns=['pergunta', 'classe', 'resposta'])

for obj in tqdm(data['data'], total=len(data['data'])):
    prompt = generate_prompt(obj['classe'],obj['resposta'])
    print(prompt)
    print("##########################################")
    completion = client.chat.completions.create(
    model=model,
    messages=[
      {"role": "system", "content": "You are helpful AI assistant."},
      {"role": "user", "content": prompt}
    ]
    )
    text = completion.choices[0].message.content
    print(text)
    
    dataset = pd.concat([dataset, pd.DataFrame([{"pergunta": text, "classe": obj['classe'], "resposta": obj['resposta']}])], ignore_index=True)


  0%|          | 0/79 [00:00<?, ?it/s]


Please generate a question that an user might be asking in order to receive the 'answer' or 'resposta' provided below.
The 'classe', or 'class' is also provided to help you out in figuring a proper question.

### Classe: Geral – Atendimento Presencial

### Resposta: A plataforma gov.br está com uma iniciativa de atendimento presencial em parceria com algumas unidades de atendimento de Estados e Municípios, chamada de Balcão gov.br. Nestes locais, o cidadão poderá receber ajuda para criar sua conta gov.br, recuperar a sua senha e manter o seu acesso seguro. Saiba mais em: gov.br/presencial. Novas unidades serão credenciadas no futuro.

Answer directly with the question, don't say anything else.
    
##########################################
Quais serviços estão disponíveis nos balcões de atendimento presencial do gov.br e onde posso encontrar mais informações sobre eles?

Please generate a question that an user might be asking in order to receive the 'answer' or 'resposta' provided be

In [23]:
dataset

,pergunta,classe,resposta
0,Quais serviços estão disponíveis nos balcões d...,Geral – Atendimento Presencial,A plataforma gov.br está com uma iniciativa de...
1,Qual suporte vocês oferecem para problemas enc...,Geral - Problema em sistemas diversos,Escrevemos em atenção à sua solicitação para i...
2,Como faço para obter um arquivo com o registro...,Geral - Arquivo da conversa no chat,"Para solicitar o arquivo com esta conversa, pe..."
3,Qual é o número de telefone para atendimento s...,Geral - Atendimento telefônico,Não há canal para atendimento telefônico sobre...
4,Como posso deixar uma avaliação positiva sobre...,Geral - Elogios,Agradecemos o seu elogio! A equipe gov.br está...
...,...,...,...
74,Como faço para acessar os serviços de Imposto ...,Nível insuficiente - Imposto de Renda,Os serviços de Imposto de Renda (https://www.g...
75,Como um MEI pode acessar e melhorar o nível da...,Nível insuficiente - MEI,Os serviços de MEI são prestados pela Receita ...
76,Como posso acessar os serviços do Meu INSS se ...,Nível insuficiente - Meu INSS,Os serviços do Meu INSS (https://meu.inss.gov....
77,Como posso acessar e utilizar o sistema Regist...,Nível insuficiente - Registrato,O sistema Registrato (https://registrato.bcb.g...


In [25]:
dataset.to_csv("scripts_dataset.csv", index=False)